In [1]:
!wget https://he-s3.s3.amazonaws.com/media/hackathon/brainwaves-machine-learning/clustering-financial-news/9379d3dc-0-dataset.tar.gz
  

--2019-01-25 04:09:49--  https://he-s3.s3.amazonaws.com/media/hackathon/brainwaves-machine-learning/clustering-financial-news/9379d3dc-0-dataset.tar.gz
Resolving he-s3.s3.amazonaws.com (he-s3.s3.amazonaws.com)... 52.219.40.128
Connecting to he-s3.s3.amazonaws.com (he-s3.s3.amazonaws.com)|52.219.40.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2962837 (2.8M) [application/x-tar]
Saving to: ‘9379d3dc-0-dataset.tar.gz’

9379d3dc-0-dataset. 100%[===================>]   2.83M  2.12MB/s    in 1.3s    

2019-01-25 04:09:51 (2.12 MB/s) - ‘9379d3dc-0-dataset.tar.gz’ saved [2962837/2962837]



In [2]:
!tar -xvzf 9379d3dc-0-dataset.tar.gz


dataset/sample_submission.zip
dataset/news.csv
dataset/


In [108]:
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')
from textblob import Word
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [308]:
data = pd.read_csv('/content/dataset/news.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 3 columns):
id          3000 non-null object
headline    3000 non-null object
text        3000 non-null object
dtypes: object(3)
memory usage: 70.4+ KB


In [309]:
data.head(5)

,id,headline,text
0,uid-1,Market Advances 5.12 More,NEW YORK (AP) - A prime rate reduction by New ...
1,uid-2,District Boosts Request For Anti-Terrorism Aid...,Mayor Anthony A. Williams petitioned the White...
2,uid-3,"Election? Here's How You Do It, Mate.",From our downunder perspective here in Austral...
3,uid-4,The Biggest Boom Ever,We are about to rewrite history. Unless a rece...
4,uid-5,Economic Aide Sees Uptrend,"Sedate and scholarly Dr. Arthur Burns, the ex-..."


In [310]:
data.iloc[0][1]

'Market Advances 5.12 More'

In [0]:
#from html.parser import HTMLParser


In [0]:
#parser = HTMLParser.HTMLParser()


In [0]:
import re
def striphtml(data):
    p = re.compile(r'<.*?>')
    return p.sub('', data)


In [0]:
data['text'] = data['text'].apply(lambda x : striphtml(x))

In [313]:
data['text'][0]

"NEW YORK (AP) - A prime rate reduction by New York's First National City Bank helped the stock market close out the busiest week in its history with a modest advance Friday.The Dow Jones average of 30 industrial stocks gained 5.12 to 929.63, finishing the week withanetrise of 18.50 points.Big Board volume, at 25.94 million shares, fell a good deal short of matching Thursday's all-time high of 38.45 million.But the day�۪s figure was more than enough to push the total for the week to a record 159.69 million shares. The previous peak had been 145.68 million in the last week of January 1975.The Dow has advanced in all but two of the first 11 sessions of the new year. As of Friday�۪s close it had gained 77.22 points since New Year�۪s."

In [0]:
# preprocessing of headline and text column
preprocess = ['headline', 'text']
stop = stopwords.words('english')
for column in preprocess:
    # to lowercase
    data[column] = data[column].apply(lambda x: " ".join(x.lower() for x in x.split()))
    # remove punctuation
    data[column] = data[column].str.replace('[^\w\s]','')
    # remove stop words
    data[column] = data[column].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

In [0]:
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(data['text'])

In [316]:
X

<3000x43739 sparse matrix of type '<class 'numpy.float64'>'
	with 299399 stored elements in Compressed Sparse Row format>

In [0]:
X = X.toarray()

In [318]:
X.shape

(3000, 43739)

In [319]:
type(X)

numpy.ndarray

In [0]:
df = pd.DataFrame(X,columns = vectorizer.get_feature_names())

In [321]:
df.columns

Index(['00', '000', '00000', '0001', '001', '002', '00217', '003', '005',
       '006',
       ...
       'zooming', 'zooms', 'zopacom', 'zuckerberg', 'zuhdi', 'zurich', 'zweng',
       'zyuganov', 'ʉi', 'ωt'],
      dtype='object', length=43739)

In [0]:
Y = vectorizer.fit_transform(data['headline'])

In [323]:
Y

<3000x5957 sparse matrix of type '<class 'numpy.float64'>'
	with 20307 stored elements in Compressed Sparse Row format>

In [0]:
Y = Y.toarray()

In [325]:
type(Y)

numpy.ndarray

In [0]:
df_2 = pd.DataFrame(Y,columns = vectorizer.get_feature_names())

In [352]:
df_2.head(1)

,01,02,022,03,04,05,064,07,08,09,...,zarb,zaslow,zero,zerocoupon,zigzags,zipping,zone,zones,zooming,ʉʉ
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:
final_df = pd.concat([df,df_2],axis=1)

In [354]:
final_df.isnull().values.any()


False

In [355]:
final_df.shape

(3000, 49696)

In [0]:
X = final_df.values

In [357]:
X.shape

(3000, 49696)

In [0]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 10)
X = pca.fit_transform(X)


In [359]:
X.shape

(3000, 10)

In [0]:
explained_variance = pca.explained_variance_ratio_

In [361]:
explained_variance

array([0.00767905, 0.00577724, 0.00535542, 0.00452894, 0.00437158,
       0.00364288, 0.00336834, 0.00291712, 0.00285517, 0.00255639])

In [362]:
true_k = 2
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)
model.fit(X)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=100,
    n_clusters=2, n_init=1, n_jobs=None, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [0]:
# data['cluster'] = model.predict(pca.transform((vectorizer.transform(data['text'])).toarray()))
data['cluster'] = model.predict(X)
#y_means = model.predict(X)

In [364]:
data['cluster'].value_counts()

0    2271
1     729
Name: cluster, dtype: int64

In [0]:
submission = data[['id','cluster']]


In [0]:
submission.to_csv('result_final_csv_2.csv')

In [0]:
x = vectorizer.transform(data['headline'])

In [0]:
x = x.toarray()

In [0]:
x = pca.transform(x)

In [0]:
np.savetxt('result_final_2.txt', X, delimiter=" ")

In [146]:
x.shape

(3000, 500)

In [62]:
data['cluster'].value_counts()

2    2419
1     182
0     179
3     120
4     100
Name: cluster, dtype: int64

In [0]:
df = pd.read_csv('/content/result.csv')

In [89]:
df.head()

,Unnamed: 0,id,cluster
0,0,uid-1,3
1,1,uid-2,1
2,2,uid-3,1
3,3,uid-4,1
4,4,uid-5,1


In [90]:
df.shape

(3000, 3)

In [26]:
df.columns

Index(['Unnamed: 0', 'id', 'cluster'], dtype='object')

In [0]:
df.drop('Unnamed: 0', axis=1, inplace=True)


In [92]:
df.columns

Index(['id', 'cluster'], dtype='object')

In [93]:
df.head()

,id,cluster
0,uid-1,3
1,uid-2,1
2,uid-3,1
3,uid-4,1
4,uid-5,1


In [0]:
df.set_index('id',inplace=True)

In [31]:
df.head()

,cluster
id,
uid-1,1
uid-2,1
uid-3,1
uid-4,1
uid-5,1


In [32]:
df.columns

Index(['cluster'], dtype='object')

In [33]:
df['cluster'].value_counts()

1    2781
0     219
Name: cluster, dtype: int64

In [0]:
df.to_csv('final_sub_a.csv')